# Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [1]:
# Warning control
import warnings
from crewai import Agent, Task, Crew
warnings.filterwarnings('ignore')

/Users/akaoui/Desktop/Github/ml-projects/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:898: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [2]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [3]:
# CReating Agents 

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [5]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [6]:
directory_read_tool = DirectoryReadTool(directory='./tools/instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [7]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.

In [8]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        #  custom code tool goes here
        return "positive"
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [9]:

lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [10]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [11]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

In [12]:
inputs = {
    "lead_name": "DataChain",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Chihab AKa",
    "position": "Crypto Trader",
    "milestone": "product AI Bot Trading launch"
}


result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DataChain, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DataChain to create a comprehensive report.

Action: Read a file's content
Action Input: {"file_path": "DataChain_company_profile.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: 'DataChain_company_profile.txt'.
 Tool Read a file's content accepts these inputs: Read a file's content(file_

2025-04-20 01:43:24,665 - 4570183168 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--238e719eac5662db5098cba0dca71e6151614e165b3e7527dc06a607990e1e1c


Thought: I need to gather as much information as possible about DataChain to create a comprehensive report.

Action: Search the internet
Action Input: {"search_query": "DataChain company profile Tokyo"} 


Search results: Title: Datachain (Tokyo) - Valuation, Funding & Investors - PitchBook
Link: https://pitchbook.com/profiles/company/530124-76
Snippet: Datachain (Tokyo) General Information  The company offers a wide range of products that connect public and permissioned blockchains as well as traditional systems, thereby enabling data from around the world to be connected securely by blockchain.
---
Title: Datachain, Inc. | Enabling the World to Interact Transparently as One ...
Link: https://www.datachain.jp/
Snippet: Datachain is a tech company that revolutionizes blockchain interoperability and stablecoin settlement. we aim to make it possible for the world to interact ...
---
Title: Datachain - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/data

In [13]:
from IPython.display import Markdown
Markdown(result)

Subject: Enhancing Data Connectivity with CrewAI's Solutions at DataChain

Dear Chihab AKa,

I hope this message finds you well. I am reaching out to you on behalf of CrewAI, a leading provider of innovative solutions in workflow management systems and memory management. As the Crypto Trader at DataChain, I understand the importance of secure data connections and the need for efficient performance optimization within your applications.

I wanted to congratulate you on the recent launch of the AI Bot Trading product at DataChain. Your dedication to enhancing data connectivity securely through blockchain technology is truly commendable. As a core contributor of YUI, facilitating interoperability between multiple ledgers through Cosmos IBC, DataChain's commitment to revolutionizing the way data is securely connected by blockchain aligns perfectly with CrewAI's mission to support businesses like yours in achieving their goals.

Our solutions in workflow management systems can complement DataChain's focus on enhancing collaboration and task assignments, ensuring efficient execution and streamlined processes. Additionally, our expertise in memory management can optimize DataChain's performance by efficiently managing memory resources within your applications, preventing memory-related issues and enhancing overall user experience.

I believe that by leveraging CrewAI's solutions, we can further strengthen DataChain's platform for data distribution, emphasizing the importance of secure and transparent interactions. Our tailored engagement strategies around blockchain interoperability and secure data connections can drive mutual growth and solidify our partnership for future success.

I would love the opportunity to discuss how CrewAI can support DataChain in achieving its objectives and enhancing data connectivity securely through innovative solutions. Please let me know a convenient time for a meeting or call to explore the possibilities further.

Thank you for your time, Chihab. I look forward to the opportunity to collaborate with you and support DataChain in its mission to revolutionize the way data is securely connected by blockchain.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

[Contact Information]